In [ ]:
import numpy as np
import matplotlib.pyplot as plt
# from cellpose import models
from cellpose.io import imread
import glob
from pathlib import Path
from PIL import Image, ImageSequence
from tqdm import tqdm
import os
import os.path
# from livecellx import segment
from livecellx import core
from livecellx.core import datasets
from livecellx.core.datasets import LiveCellImageDataset, SingleImageDataset
from skimage import measure
from livecellx.core import SingleCellTrajectory, SingleCellStatic
# import detectron2
# from detectron2.utils.logger import setup_logger

# setup_logger()

# import some common libraries
import numpy as np
import os, json, cv2, random
import cv2

## Load model

In [ ]:
import torch
from pathlib import Path
# model_dir = Path(r"./scripts/mmdetection_classify/work_dirs/tsn_imagenet-pretrained-r50_8xb32-1x1x3-100e_kinetics400-rgb-v6-combined-clipL=2-CLIP_NUM=4/")
# model_dir = Path(r"./scripts/mmdetection_classify/work_dirs/tsn_imagenet-pretrained-r50_8xb32-1x1x3-100e_kinetics400-rgb-v8-combined-clipLen=2-trainClipNum=3-valClipNum=3")
# model_dir = Path(r"./scripts/mmdetection_classify/work_dirs/tsn_imagenet-pretrained-r50_8xb32-1x1x3-100e_kinetics400-rgb-v8-combined-clipLen=1-trainClipNum=3-valClipNum=3")
# path = r"./scripts/mmdetection_classify/work_dirs/tsn_imagenet-pretrained-r50_8xb32-1x1x3-100e_kinetics400-rgb-v6-combined-clipL=2/best_acc_top1_epoch_28.pth"
# model = torch.load(path)
# model_dir = Path(r"./scripts/mmdetection_classify/work_dirs/tsn_imagenet-pretrained-r50_8xb32-1x1x3-100e_kinetics400-rgb-v9-combined-clipLen=3-trainClipNum=3-valClipNum=3")
# model_dir = Path(r"./scripts/mmdetection_classify/work_dirs/tsn_imagenet-pretrained-r50_8xb32-1x1x3-100e_kinetics400-rgb-v10-st-combined-clipLen=3-trainClipNum=3-valClipNum=3")
# model_dir = Path(r"./scripts/mmdetection_classify/work_dirs/tsn_imagenet-pretrained-r50_8xb32-1x1x3-100e_kinetics400-rgb-v10-drop-div-combined-clipLen=2-trainClipNum=3-valClipNum=3")
# model_dir = Path(r"./scripts/mmdetection_classify/work_dirs/tsn_imagenet-pretrained-r50_8xb32-1x1x3-100e_kinetics400-rgb-v12-st-combined-clipLen=2-trainClipNum=3-valClipNum=3/")
# out_dir = Path(r"./scripts/mmdetection_classify/work_dirs/test_results/v12-st-combined-clipLen=2-trainClipNum=3")

# model_dir = Path(r"./scripts/mmdetection_classify/work_dirs/tsn_imagenet-pretrained-r50_8xb32-1x1x3-100e_kinetics400-rgb-v12-st-combined-clipLen=2-trainClipNum=3-valClipNum=3/")
# out_dir = Path(r"./scripts/mmdetection_classify/work_dirs/test_results/v12-st-combined-clipLen=2-trainClipNum=3")
model_dir = Path(r"./scripts/mmdetection_classify/work_dirs/tsn_imagenet-pretrained-r50_8xb32-1x1x3-100e_kinetics400-rgb-v12-st-video-clipLen=2-trainClipNum=3-valClipNum=3/")
out_dir = Path(r"./scripts/mmdetection_classify/work_dirs/test_results/v12-st-video-clipLen=2-trainClipNum=3-epoch=18")

out_dir.mkdir(parents=True, exist_ok=False)

Read a trained MMAction model


In [ ]:
from mmengine.config import Config, DictAction
from mmaction.registry import MODELS
# config_file = str(model_dir / "tsn_imagenet-pretrained-r50_8xb32-1x1x3-100e_kinetics400-rgb.py")
# config_file = str(model_dir / "config_train_v12_st_cliplen=2_clipnum=3-combined.py")
config_file = str(model_dir / "config_train_v12_st_cliplen=2_clipnum=3-video.py")
# checkpoint_file = str(model_dir / "best_acc_top1_epoch_5.pth")
# checkpoint_file = str(model_dir / "epoch_52.pth")
# checkpoint_file = str(model_dir / "best_acc_top1_epoch_126.pth")
# checkpoint_file = str(model_dir / "epoch_250.pth")
# checkpoint_file = str(model_dir / "best_acc_top1_epoch_448.pth")
# checkpoint_file = str(model_dir / "epoch_450.pth")
checkpoint_file = str(model_dir / "best_acc_top1_epoch_18.pth")

cfg = Config.fromfile(config_file)

In [ ]:
# model = MODELS.build()
import mmcv
from mmaction.apis import init_recognizer, inference_recognizer

DEVICE = 'cuda:0' # or 'cpu', based on your device
model = init_recognizer(config_file, checkpoint_file, device=DEVICE)

Load test dataframe and evaluate.

In [ ]:
import pandas as pd
# video_dir = r"./notebook_results/mmaction_train_data_v5/videos"
# mmaction_data_tsv = r"./notebook_results/mmaction_train_data_v6/test_data_combined.txt"
# video_dir = r"./notebook_results/mmaction_train_data_v8/videos"
# mmaction_data_tsv = r"./notebook_results/mmaction_train_data_v8/mmaction_test_data_combined.txt"
# mmaction_data_tsv = r"./notebook_results/mmaction_train_data_v8/mmaction_train_data_combined.txt"
# video_dir = r"./notebook_results/mmaction_train_data_v9/videos"
# mmaction_data_tsv = r"./notebook_results/mmaction_train_data_v9/mmaction_test_data_combined.txt"
# video_dir = r"./notebook_results/mmaction_train_data_v10-st/videos"
# mmaction_data_tsv = r"./notebook_results/mmaction_train_data_v10-st/mmaction_test_data_combined.txt"
# video_dir = r"./notebook_results/mmaction_train_data_v10-drop-div/videos"
# mmaction_data_tsv = r"./notebook_results/mmaction_train_data_v10-drop-div/mmaction_test_data_combined.txt"

video_dir = r"./notebook_results/mmaction_train_data_v12-st/videos"
mmaction_data_tsv = r"./notebook_results/mmaction_train_data_v12-st/mmaction_test_data_combined.txt"
test_data_meta_path = r"./notebook_results/mmaction_train_data_v12-st/test_data.txt"


In [ ]:
# test dataframe
test_data_df = pd.read_csv(test_data_meta_path, sep=" ")
test_data_df = test_data_df.rename(columns={"label_index": "label"})
print(test_data_df.shape)
test_data_df[:2]

In [ ]:
# mmaction df columns are path and label
mmaction_data_df = pd.read_csv(mmaction_data_tsv, sep=" ", header=None)
mmaction_data_df.columns = ["path", "label"]
print(mmaction_data_df.shape)
mmaction_data_df[:2]

In [ ]:
from tqdm import tqdm
# # test on all video in the df
# test_video_paths = []
# test_gt_labels = []
# for i, row in tqdm(test_data_df.iterrows()):
#     video_path = os.path.join(video_dir, row["path"])
#     test_video_paths.append(video_path)
#     test_gt_labels.append(row["label"])

In [ ]:
nclasses = 3
gt2total = {class_idx: 0 for class_idx in range(nclasses)}
gt2correct = {class_idx: 0 for class_idx in range(nclasses)}
wrong_predictions = []
all_predictions = []
video_dir = Path(video_dir)
for row_ in tqdm(test_data_df.iterrows()):
    idx, row_series = row_
    video_path = str(video_dir / row_series["path"])
    try:
        results = inference_recognizer(model, video_path)
    except Exception as e:
        print("exception during prediction:", e)
        continue
    predicted_label = results.pred_labels.item.cpu().numpy()[0]
    test_gt_label = row_series["label"]
    if test_gt_label not in gt2total:
        gt2total[test_gt_label] = 0
        gt2correct[test_gt_label] = 0
    gt2total[test_gt_label] += 1
    row_series = row_series.copy()
    row_series["predicted_label"] = predicted_label
    row_series["true_label"] = test_gt_label
    row_series["correct"] = predicted_label == test_gt_label
    all_predictions.append(row_series)
    if predicted_label != test_gt_label:
        print("wrong prediction:", video_path, "predicted_label:", predicted_label, "gt_label:", test_gt_label)
        wrong_predictions.append(row_series)
    else:
        gt2correct[test_gt_label] += 1


In [ ]:
for test_gt_label, total in gt2total.items():
    correct = gt2correct[test_gt_label]
    if total == 0:
        print("no video for label:", test_gt_label)
        continue
    print("gt_label:", test_gt_label, "total:", total, "correct:", correct, "acc:", correct / total)

In [ ]:
# convert all series in wrong_predictions to dataframe
all_predictions_df = pd.DataFrame(all_predictions)
wrong_predictions_df = pd.DataFrame(wrong_predictions)
wrong_predictions_df[:2]

In [ ]:
from sklearn.metrics import classification_report, accuracy_score, precision_score, f1_score

def report_classification_metrics(true_labels, predicted_labels):
    # calculate the accuracy
    accuracy = accuracy_score(true_labels, predicted_labels)

    # calculate the precision
    precision = precision_score(true_labels, predicted_labels, average="weighted")

    # calculate the F1 score
    f1 = f1_score(true_labels, predicted_labels, average="weighted")

    # generate a classification report
    report = classification_report(true_labels, predicted_labels)

    # print the metrics and classification report
    print(f"Accuracy: {accuracy:.2f}")
    print(f"Precision: {precision:.2f}")
    print(f"F1 Score: {f1:.2f}")
    print("Classification Report:")
    print(report)
report_classification_metrics(all_predictions_df["true_label"], all_predictions_df["predicted_label"])

In [ ]:
# all_predictions_df.to_csv(out_dir/"all_predictions.csv", index=False)
# wrong_predictions_df.to_csv(out_dir/"wrong_predictions.csv", index=False)
wrong_predictions_df = pd.read_csv(out_dir/"wrong_predictions.csv")


### Visualize wrong predictions according to three frame types: combined, raw and mask

In [ ]:
# Extract the last three levels of the src_dir path
wrong_predictions_df['short_src_dir'] = wrong_predictions_df['src_dir'].str.split(r'\\|/').apply(lambda x: '/'.join(x[-3:]))

In [ ]:
import seaborn as sns


# Set the font scale to 2
sns.set(font_scale=2)

# Unique frame types
unique_frame_types = wrong_predictions_df['frame_type'].unique()

# Create visualizations for each frame type
for frame_type in unique_frame_types:
    filtered_df = wrong_predictions_df[wrong_predictions_df['frame_type'] == frame_type]
    
    # Set up the figure and axes
    fig, axs = plt.subplots(2, 2, figsize=(18, 14))
    fig.suptitle(f'Distribution of Wrong Predictions for Frame Type: {frame_type}', fontsize=40)

    # Plotting the distribution of 'label'
    sns.countplot(data=filtered_df, x='label', ax=axs[0, 0])
    axs[0, 0].set_title('Label Distribution')

    # Plotting the distribution of 'short_src_dir'
    sns.countplot(data=filtered_df, y='short_src_dir', ax=axs[0, 1], order=filtered_df['short_src_dir'].value_counts().index)
    axs[0, 1].set_title('Updated Source Directory Distribution (3 Levels)')
    axs[0, 1].tick_params(axis='y', rotation=0)

    # Plotting the distribution of 'padding_pixels'
    sns.countplot(data=filtered_df, x='padding_pixels', ax=axs[1, 0])
    axs[1, 0].set_title('Padding Pixels Distribution')

    # Placeholder for potential other visualizations for different frame types
    axs[1, 1].axis('off')

    plt.tight_layout(rect=[0, 0.03, 1, 0.95])
    plt.show()